Here's my attempt to visualize the input data next to its target in a convienient interactive way with **plotly** and **ipywidgets**.

This notebook requires an active kernel for processing different **stock_id** and **time_id** data, so click "Copy and Edit" to play with data selection dropdown lists. I believe it's possible to rewrite the code and acquire preprocessed data with **plotly** dropdown menus instead of **ipywidgets**, but it seems like too much work for now.

Enjoy!

# Data preparation

In [ ]:
import ipywidgets as widgets

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

import pandas as pd
import numpy as np

In [ ]:
data_dir = '../input/optiver-realized-volatility-prediction/'

def preprocessor(stock_id):   
    file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
    file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
    
    book_df = pd.read_parquet(file_path_book)
    trade_df = pd.read_parquet(file_path_trade)
    
    return book_df, trade_df

In [ ]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def log_return(series):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [ ]:
train_df = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
df_stock_ids = train_df['stock_id'].unique()
df_time_ids = train_df[train_df['stock_id'] == 0].time_id.unique()
book_df, trade_df = preprocessor(stock_id=0)

In [ ]:
stock_selector = widgets.Dropdown(options=df_stock_ids, description='stock_id')
time_selector = widgets.Dropdown(options=df_time_ids, description='time_id')

def update_stock_data(*args):
    global book_df, trade_df
    book_df, trade_df = preprocessor(stock_selector.value)
    time_selector.options = book_df.time_id.unique()
    
stock_selector.observe(update_stock_data, 'value')

## Plotting function:

In [ ]:
def plot_stock_by_time_plotly(stock_id, time_id, price_shape='hv', volume_shape='hv'):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights = [3, 1, 1], vertical_spacing=0.04, 
                        subplot_titles=['Price', 'Volume', 'Realized Volatility'])
    price_colors = ['green', 'lime', 'pink', 'red']
    
    # Book data
    selected_book_data = book_df[book_df.time_id == time_id].copy()
    selected_book_data['wap1'] = calc_wap1(selected_book_data)
    size_data = selected_book_data[['bid_size2', 'bid_size1', 'ask_size1', 'ask_size2']]

    x = selected_book_data.seconds_in_bucket
    for ci, col in enumerate(['bid_price2', 'bid_price1', 'ask_price1', 'ask_price2']):
        size_info = size_data.iloc[:, ci].astype(str)
        fig.add_scatter(x=x, y=selected_book_data[col], name=col, legendgroup = 'price', line_shape=price_shape, customdata=size_info, row=1, col=1,
                        hovertemplate='%{y:.6f}, size: %{customdata}')
    fig.add_scatter(x=x, y=selected_book_data['wap1'], name='wap1', legendgroup='price', line_shape=price_shape,
                    line=dict(color='blue', width=0.5), row=1, col=1)

    for ci, col in enumerate(['bid_size2', 'bid_size1', 'ask_size1', 'ask_size2']):
        fig.add_scatter(x=x, y=selected_book_data[col], name=col, stackgroup='volume', legendgroup='volume', line=dict(width=0),
                        fillcolor=price_colors[ci], line_shape=volume_shape, row=2, col=1)
    
    # Volatility
    min_periods_for_volty = 20
    selected_book_data['actual_realzd_volty'] = selected_book_data['wap1'].agg(log_return).expanding(min_periods=min_periods_for_volty).apply(realized_volatility)
    selected_book_data['weighted_realzd_volty'] = selected_book_data['actual_realzd_volty'] * np.sqrt(600 / selected_book_data['seconds_in_bucket'])
    target = train_df.query('stock_id == @stock_id & time_id == @time_id').target.item()

    fig.add_scatter(x=x, y=selected_book_data['actual_realzd_volty'], name='real_volty', legendgroup='volatility', line_color='blue', row=3, col=1)
    fig.add_scatter(x=x, y=selected_book_data['weighted_realzd_volty'], name='w_real_volty', legendgroup='volatility', line_color='purple', row=3, col=1)
    fig.add_scatter(x=[601], y=[target], name='target', legendgroup='volatility', line_color='red', row=3, col=1)

    # Trade data
    selected_trade_data = trade_df[trade_df.time_id == time_id]
    x = selected_trade_data.seconds_in_bucket
    trade_size_info = selected_trade_data.loc[:, 'size'].astype(str)
    fig.add_scatter(x=x, y=selected_trade_data['price'], name=f'trade_price', mode='markers', line_color='brown', marker_size=6, legendgroup='price',
                   customdata=trade_size_info, hovertemplate='%{y:.6f}, size: %{customdata}')
    fig.add_bar(x=x, y=-selected_trade_data['size'], name=f'trade_size', marker_color='brown', legendgroup = 'volume', row=2, col=1, 
                customdata=trade_size_info, hovertemplate='%{customdata}')
    
    # Line shape (interpolation) menus
    menu_x = 1.015
    def line_shape_menu(traces, options, order):
        return dict(buttons=[dict(args=[{"line.shape": op}, traces], label=op_label, method='restyle') for op, op_label in options],
                    direction="down", pad={"r": 10, "t": 0}, showactive=True, x=menu_x, y=0.7-0.4*order, xanchor="left", yanchor="top")
    updatemenus = [line_shape_menu(traces, [('hv', 'HV (actual)'), ('linear', 'Linear')], i) 
                   for i, traces in enumerate([[0, 1, 2, 3, 4], [5, 6, 7, 8]])]
    for order, menu_label, in enumerate(['Price interpolation', 'Volume interpolation']):
        fig.add_annotation(**dict(text=menu_label, x=menu_x, y=0.7-0.4*order, xref='paper', yref='paper', xanchor='left', yanchor="top", yshift=20, showarrow=False))
    
    # Title and layout updates
    book_recs, trade_recs = selected_book_data.shape[0], selected_trade_data.shape[0]
    fig.update_layout(title=dict(text=f'stock_id={stock_id}, time_id={time_id}  |  book records: {book_recs}, trade_records: {trade_recs}', 
                                 font_size=15, x=0.15, xanchor='left'), 
                      height=600, margin=dict(l=20, r=20, t=40, b=20), legend_tracegroupgap=124,
                      colorway=price_colors, 
                      updatemenus=updatemenus,
                      hovermode="x unified", hoverdistance=1
                     )
    fig.update_annotations(dict(x=0.01, xanchor='left', font_size=12, ), selector={'x': 0.5})    # Subplot titles moved to the left
    fig.update_xaxes(range=[-5, 606], title=dict(text='seconds_in_bucket', font_size=10, standoff=0))
    fig.update_traces(xaxis='x3')
    fig.show()

# Charts

In [ ]:
# This cell is purely for Viewer Mode demonstration of what a chart looks like. 
# Delete it when editing the notebook to remove duplicate chart and speed up plotting.

# This line makes plotly chart visible in Viewer mode, but slows down plotting in kernel:
plotly.offline.init_notebook_mode(connected=True) 
plot_stock_by_time_plotly(stock_id=0, time_id=5)

In [ ]:
# This cell displays the chart along with dropdown lists for selecting stock_id and time_id via ipywidgets.
# However, it works only when running the kernel, so the outputs are invisible in Viewer Mode
ui1 = widgets.HBox([stock_selector, time_selector])
out = widgets.interactive_output(plot_stock_by_time_plotly, dict(stock_id=stock_selector, time_id=time_selector));
display(ui1, out)

The **Price** subplot is pretty much self-explanatory. Hover to see corresponding volume sizes as well.

On the **Volume** subplot, above zero are stacked volumes from the book (adding up to a total volume on two closest levels), while the trade sizes are looking in the opposite direction.

**Realized Volatility** is displayed in two ways: 
1. As is, growing monotonously with expanding window (hardcoded minimum values for calculation: 20)
2. Multiplied by a сoefficient of $\sqrt{\frac{600}{seconds\_in\_bucket}}$ to adjust for time window duration.

---
If you have any suggestions on how to improve this visualization, please share in the comments!